<a href="https://colab.research.google.com/github/Marwanelhanafey/Tensorflow-and-PyTorch/blob/main/Advanced_Lap_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize input data
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [90]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [91]:
import time

start = time.time()
model.fit(x_train, y_train, epochs=5, batch_size=64)
end = time.time()

print(f"TensorFlow training time: {end - start:.2f} seconds")


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8374 - loss: 0.5839
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9471 - loss: 0.1918
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9606 - loss: 0.1369
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9681 - loss: 0.1101
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9744 - loss: 0.0913
TensorFlow training time: 22.20 seconds


In [92]:
# Evaluate on test set
start = time.time()
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
end = time.time()

print(f"TensorFlow Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"TensorFlow Inference Time: {end - start:.2f} seconds")


TensorFlow Test Accuracy: 96.80%
TensorFlow Inference Time: 0.98 seconds


In [95]:
import tensorflow as tf
import numpy as np
import pathlib

# ── 2. Convert to TensorFlow Lite ───────────────────────────────────────────────
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]        # optional
tflite_model = converter.convert()

tflite_path = pathlib.Path("model.tflite")
tflite_path.write_bytes(tflite_model)
print(f"✓ Saved TFLite model → {tflite_path.resolve()}")




Saved artifact at '/tmp/tmpdnt2s6r9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor_20')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140320720578576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140316703914704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140316703910672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140316703904912: TensorSpec(shape=(), dtype=tf.resource, name=None)
✓ Saved TFLite model → /content/model.tflite


In [96]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Transform: normalize to [0,1]
transform = transforms.ToTensor()

# Load data
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000)



In [97]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten
        x = F.relu(self.fc1(x))
        return self.fc2(x)



In [99]:
import torch.optim as optim
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

start = time.time()
for epoch in range(5):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * data.size(0)  # sum up batch loss
        _, predicted = output.max(1)
        correct += predicted.eq(target).sum().item()
        total += target.size(0)

    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total

    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}, Accuracy = {epoch_acc:.2f}%")

end = time.time()
print(f"PyTorch training time: {end - start:.2f} seconds")


Epoch 1: Loss = 0.3962, Accuracy = 89.38%
Epoch 2: Loss = 0.1967, Accuracy = 94.40%
Epoch 3: Loss = 0.1456, Accuracy = 95.82%
Epoch 4: Loss = 0.1176, Accuracy = 96.61%
Epoch 5: Loss = 0.0982, Accuracy = 97.17%
PyTorch training time: 93.09 seconds


In [100]:
model.eval()  # Set model to evaluation mode
correct = 0
total = 0
inference_start = time.time()

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

inference_end = time.time()
accuracy = correct / total * 100

print(f"PyTorch Test Accuracy: {accuracy:.2f}%")
print(f"PyTorch Inference Time: {inference_end - inference_start:.2f} seconds")


PyTorch Test Accuracy: 96.56%
PyTorch Inference Time: 1.29 seconds


In [101]:
# Install ONNX
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 31.5 MB/s eta 0:00:00


In [104]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

In [106]:
import torch

model.eval()

# ── 2. Prepare dummy input (batch_size=1, 784 features)
dummy_input = torch.randn(1, 784)   # adjust if your net expects a different shape

# ── 3. Export to ONNX
torch.onnx.export(
    model,                    # model to export
    dummy_input,              # example input
    "model.onnx",             # file name
    input_names  = ["input"],
    output_names = ["output"],
    opset_version=14,         # good default; change only if needed
    do_constant_folding=True  # folds batch‑norm & constants for speed
)



✓ Saved ONNX model → model.onnx
onnxruntime not installed – skip runtime test
